In [1]:
import os
import sys; sys.path.insert(0, os.path.abspath("../"))

from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
import SimpleITK as sitk
from lungmask import mask
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import numba
from general_utils.utils import plot3, parse_points_reg, compute_TRE, convert_nda_to_itk, convert_itk_to_nda, register_image_w_mask, register_image_external, normalize_copd_to_HU
from general_utils.utils import save_pts, save_pts_itk, transform_points
import torch
from skimage.exposure import rescale_intensity, equalize_hist, equalize_adapthist

sitk.ProcessObject.SetGlobalWarningDisplay(False)
sitk.ProcessObject.SetGlobalDefaultDebug(False)

this_path = Path().resolve().parent

In [2]:
# define train path
# make sure you have created the results and paramMaps directories. In paramMaps folder, save the desired external parameter fiiles (e.g. params0011)

kp_path = this_path/'data/copd/keypoints'
scans_path = this_path/'data/copd/scans/'
lung_masks_path = this_path/'data/copd/lungMasks/'
results_path = this_path/'data/copd/results/'
paramMaps_path = this_path/'data/copd/paramMaps/'

****Goal****


In the test, given INHALE (image and landmarks), predict EXHALE landmarks

**Originally**

INHALE -> Moving

Exhale -> Fixed

**Change order, for correct points transformation**

INHALE -> Fixed

Exhale -> Moving

In [4]:
# 3c - v2
# to save csv with results
exp_name = 'clahe_lm_bspline_p0049' 

# select which cases to run
cases = ['001', '002', '003', '004']

results = {}

# either to save the transformed inhale points
save_transformed_points = False
save_final_moving_image = False

for train_case in tqdm(cases, total=len(cases)):
    
    case_results = {}

    # read images and masks
    fixed_itk = sitk.ReadImage(str(scans_path/f'case_{train_case}_insp.nii.gz'))
    fixed_lung_mask = sitk.ReadImage(str(lung_masks_path/f'case_{train_case}_insp.nii.gz'))

    # read images and masks
    fixed_itk = sitk.ReadImage(str(scans_path/f'case_{train_case}_insp.nii.gz'))
    fixed_image = convert_itk_to_nda(fixed_itk)
    fixed_image[fixed_image<0] = 0
    fixed_image = fixed_image/np.max(fixed_image)
    clahe_fixed = equalize_adapthist(fixed_image)
    fixed_itk = convert_nda_to_itk(clahe_fixed, fixed_itk)

    fixed_lung_mask = sitk.ReadImage(str(lung_masks_path/f'case_{train_case}_insp.nii.gz'))

    moving_itk = sitk.ReadImage(str(scans_path/f'case_{train_case}_exp.nii.gz'))
    moving_image = convert_itk_to_nda(moving_itk)
    moving_image[moving_image<0] = 0
    moving_image = moving_image/np.max(moving_image)
    clahe_moving = equalize_adapthist(moving_image)
    moving_itk = convert_nda_to_itk(clahe_moving, moving_itk)

    moving_lung_mask = sitk.ReadImage(str(lung_masks_path/f'case_{train_case}_exp.nii.gz'))

    # # read points
    points_inhale = np.loadtxt(kp_path/f'case_{train_case}_insp.txt').astype(np.int16)
    points_exhale = np.loadtxt(kp_path/f'case_{train_case}_exp.txt').astype(np.int16)

    # read param 11
    pm_affine = sitk.ReadParameterFile(str(paramMaps_path/'Parameters.Par0011.affine.txt'))
    pm_bspline_1 = sitk.ReadParameterFile(str(paramMaps_path/'Parameters.Par0011.bspline1_s.txt'))
    pm_bspline_2 = sitk.ReadParameterFile(str(paramMaps_path/'Parameters.Par0011.bspline2_s.txt'))
    pm_bspline_49 = sitk.ReadParameterFile(str(paramMaps_path/'Par0049_stdT-advanced.txt'))

    # composed transformation

    moving_reg, mov_param = register_image_external(fixed_image=fixed_itk, fixed_mask=fixed_lung_mask, 
                                                    moving_image=moving_itk, moving_mask=moving_lung_mask, 
                                                    paramMaps=[pm_bspline_49], print_console=False)
                                                    
    moving_lm = register_image_w_mask(fixed_image=None, moving_image=moving_lung_mask, fixed_mask=None, moving_mask=None,
                                            transformParameterMap=mov_param, interpolator='nn')[0]
    
    points_inhale_moved = transform_points(moving_itk=moving_itk, mov_param=mov_param, 
                                            fixed_points_path=kp_path/f"case_{train_case}_insp.pts",
                                            output_dir=results_path/'affine/', train_case=train_case)

    case_results['baseline'] = compute_TRE(points_exhale, points_inhale, voxel_spacing=moving_itk.GetSpacing())
    case_results['bspline49'] = compute_TRE(points_exhale, points_inhale_moved, voxel_spacing=moving_itk.GetSpacing())
        
    print(f"Baseline TRE: {case_results['baseline']}")
    print(f"Registration (final) TRE: {case_results['bspline49']}")
    
    # if save_transformed_points:
    #     save_pts(points_inhale_moved_bs2, results_path/f'case_{train_case}_insp_moved.txt')
    # if save_final_moving_image:
    #     sitk.WriteImage(moving_reg_bs2, results_path/f'case_{train_case}_insp_moved.nii.gz')   
    #     sitk.WriteImage(moving_lm_bs2, results_path/f'lm_case_{train_case}_insp_moved.nii.gz')
     
    results[train_case] = case_results

pd.DataFrame(results).to_csv(results_path/f'{exp_name}.csv')

 25%|██▌       | 1/4 [07:51<23:34, 471.44s/it]

Baseline TRE: (26.5212958095204, 11.498190061390533)
Registration (final) TRE: (3.0378340757483633, 4.602693024022892)


 50%|█████     | 2/4 [13:20<12:55, 387.64s/it]

Baseline TRE: (21.93124759248582, 6.505994990605793)
Registration (final) TRE: (3.189642546581437, 4.776846552766394)


 75%|███████▌  | 3/4 [20:10<06:37, 397.69s/it]

Baseline TRE: (12.62588335216309, 6.381867428751379)
Registration (final) TRE: (1.1370457329310535, 1.0765540322289193)


100%|██████████| 4/4 [26:08<00:00, 392.16s/it]

Baseline TRE: (29.583559738904107, 12.92417092574431)
Registration (final) TRE: (1.6686778254419494, 1.7203474553750309)


In [5]:
# 3c - v2
# to save csv with results
exp_name = 'clahe_lm_bspline_p0049_stddT2000itr' 

# select which cases to run
cases = ['001', '002', '003', '004']

results = {}

# either to save the transformed inhale points
save_transformed_points = False
save_final_moving_image = False

for train_case in tqdm(cases, total=len(cases)):
    
    case_results = {}

    # read images and masks
    fixed_itk = sitk.ReadImage(str(scans_path/f'case_{train_case}_insp.nii.gz'))
    fixed_lung_mask = sitk.ReadImage(str(lung_masks_path/f'case_{train_case}_insp.nii.gz'))

    # read images and masks
    fixed_itk = sitk.ReadImage(str(scans_path/f'case_{train_case}_insp.nii.gz'))
    fixed_image = convert_itk_to_nda(fixed_itk)
    fixed_image[fixed_image<0] = 0
    fixed_image = fixed_image/np.max(fixed_image)
    clahe_fixed = equalize_adapthist(fixed_image)
    fixed_itk = convert_nda_to_itk(clahe_fixed, fixed_itk)

    fixed_lung_mask = sitk.ReadImage(str(lung_masks_path/f'case_{train_case}_insp.nii.gz'))

    moving_itk = sitk.ReadImage(str(scans_path/f'case_{train_case}_exp.nii.gz'))
    moving_image = convert_itk_to_nda(moving_itk)
    moving_image[moving_image<0] = 0
    moving_image = moving_image/np.max(moving_image)
    clahe_moving = equalize_adapthist(moving_image)
    moving_itk = convert_nda_to_itk(clahe_moving, moving_itk)

    moving_lung_mask = sitk.ReadImage(str(lung_masks_path/f'case_{train_case}_exp.nii.gz'))

    # # read points
    points_inhale = np.loadtxt(kp_path/f'case_{train_case}_insp.txt').astype(np.int16)
    points_exhale = np.loadtxt(kp_path/f'case_{train_case}_exp.txt').astype(np.int16)

    # read param 11
    pm_bspline_49_stdT2000 = sitk.ReadParameterFile(str(paramMaps_path/'Par0049_stdT2000itr.txt'))
    pm_bspline_49 = sitk.ReadParameterFile(str(paramMaps_path/'Par0049_stdT-advanced.txt'))

    # composed transformation

    moving_reg, mov_param = register_image_external(fixed_image=fixed_itk, fixed_mask=fixed_lung_mask, 
                                                    moving_image=moving_itk, moving_mask=moving_lung_mask, 
                                                    paramMaps=[pm_bspline_49_stdT2000], print_console=False)
                                                    
    moving_lm = register_image_w_mask(fixed_image=None, moving_image=moving_lung_mask, fixed_mask=None, moving_mask=None,
                                            transformParameterMap=mov_param, interpolator='nn')[0]
    
    points_inhale_moved = transform_points(moving_itk=moving_itk, mov_param=mov_param, 
                                            fixed_points_path=kp_path/f"case_{train_case}_insp.pts",
                                            output_dir=results_path/'affine/', train_case=train_case)

    case_results['baseline'] = compute_TRE(points_exhale, points_inhale, voxel_spacing=moving_itk.GetSpacing())
    case_results['bspline49'] = compute_TRE(points_exhale, points_inhale_moved, voxel_spacing=moving_itk.GetSpacing())
        
    print(f"Baseline TRE: {case_results['baseline']}")
    print(f"Registration (final) TRE: {case_results['bspline49']}")
    
    # if save_transformed_points:
    #     save_pts(points_inhale_moved_bs2, results_path/f'case_{train_case}_insp_moved.txt')
    # if save_final_moving_image:
    #     sitk.WriteImage(moving_reg_bs2, results_path/f'case_{train_case}_insp_moved.nii.gz')   
    #     sitk.WriteImage(moving_lm_bs2, results_path/f'lm_case_{train_case}_insp_moved.nii.gz')
     
    results[train_case] = case_results

pd.DataFrame(results).to_csv(results_path/f'{exp_name}.csv')

 25%|██▌       | 1/4 [03:40<11:00, 220.04s/it]

Baseline TRE: (26.5212958095204, 11.498190061390533)
Registration (final) TRE: (7.414896862608816, 10.845450811567758)


 50%|█████     | 2/4 [07:11<07:09, 214.94s/it]

Baseline TRE: (21.93124759248582, 6.505994990605793)
Registration (final) TRE: (3.749790446031328, 5.12335536840661)


 75%|███████▌  | 3/4 [11:00<03:41, 221.26s/it]

Baseline TRE: (12.62588335216309, 6.381867428751379)
Registration (final) TRE: (1.238285842235741, 1.1477941544170276)


100%|██████████| 4/4 [14:30<00:00, 217.59s/it]

Baseline TRE: (29.583559738904107, 12.92417092574431)
Registration (final) TRE: (28.09652172342388, 11.184796279744194)
